In [1]:
import numpy as np
import healpy as hp
import math
import time
import normsandangles
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import multiprocessing as mp
from joblib import Parallel, delayed
from pylab import arange, show, cm
import scipy
from scipy import special

%matplotlib inline
mpl.rcParams.update({'font.size':12})

#https://stackoverflow.com/questions/34023932/mollview-use-matplotlib-colormaps-and-change-background-color
cmap = cm.jet #stores old healpy colloring scheme (cmap='jet')
cmap.set_under('w')

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

## Sem paralelismo

In [45]:
def phi_of_k_cart(norms_and_vecs):

    phi_rand = []
    phi_k_dict = {}
    for norm in norms_and_vecs.keys():
        phi_rand = np.random.randn(8,len(norms_and_vecs[norm]))#we need 8 independent random vectors. See pdf for details
        phi_k_dict.update({norm:phi_rand})
    return phi_k_dict

def phi_of_x_cart(Lsize,x,y,z,norms_and_vecs,Phi_k_dict,alpha):
    
    Phi_x = 0.0 #Initial value of \Phi(\vec{x})
    prefactor = 0.0
    R = 1.0 #Radius of the CMB surface
    A = 1.0 #Amplitude of the Power Spectrum
    
    for norm in norms_and_vecs.keys():
        
        k = 2.0*math.pi*norm/Lsize/alpha#norm of Fourier vector
        sqrt_Pspec = math.sqrt(A*(k**(-3.0)))#Square-root of Harrison-Zeldovich Power-spectrum
        khat = np.transpose(norms_and_vecs[norm])# unit Fourier vectors
        
        cos_gamma1 = np.dot(np.array([x,y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the first octant        
        cos_gamma2 = np.dot(np.array([x,-y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 2nd octant: y -> - y        
        cos_gamma3 = np.dot(np.array([-x,-y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 3rd octant: x, y -> -x, -y        
        cos_gamma4 = np.dot(np.array([-x,y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 4th octant: x -> -x 
        
        prefactor = 2*(np.cos(k*R*cos_gamma1)*Phi_k_dict[norm][0]-np.sin(k*R*cos_gamma1)*Phi_k_dict[norm][1] + \
                    np.cos(k*R*cos_gamma2)*Phi_k_dict[norm][2]-np.sin(k*R*cos_gamma2)*Phi_k_dict[norm][3] + \
                    np.cos(k*R*cos_gamma3)*Phi_k_dict[norm][4]-np.sin(k*R*cos_gamma3)*Phi_k_dict[norm][5] + \
                    np.cos(k*R*cos_gamma4)*Phi_k_dict[norm][6]-np.sin(k*R*cos_gamma4)*Phi_k_dict[norm][7])
        prefactor = np.sum(prefactor) #sum over all angles with fixed nnorm
        
        Phi_x += sqrt_Pspec*prefactor #sum over norms
    return Phi_x

def phi_map_cart(Nside,nmax,Lsize,alpha,seed):
    np.random.seed(seed) # fixes random seed, for debugging
    normvecs_dict = normsandangles.vector_n(nmax,alpha) #initialize norms and vectors
    randphi_dict = phi_of_k_cart(normvecs_dict) #initialize random values of \phi(\vec{k})
    phi_in_pixel = []
    pixels = np.arange(hp.nside2npix(Nside))
    for x,y,z in np.transpose(hp.pix2vec(Nside,pixels)):
        phi_in_pixel.append(phi_of_x_cart(Lsize,x,y,z,normvecs_dict,randphi_dict,alpha))
    return np.array(phi_in_pixel)

In [46]:
%%time
mapa1 = phi_map_cart(64,6,0.1,4,1)

CPU times: user 3min 42s, sys: 3.66 ms, total: 3min 42s
Wall time: 3min 42s


## Com paralelismo

In [47]:
def phi_of_x_cart(Lsize,x,y,z,norms_and_vecs,Phi_k_dict,alpha):
    Phi_x = 0.0 #Initial value of \Phi(\vec{x})
    prefactor = 0.0
    R = 1.0 #Radius of the CMB surface
    A = 1.0 #Amplitude of the Power Spectrum
    
    for norm in norms_and_vecs.keys():
        
        k = 2.0*math.pi*norm/Lsize/alpha#norm of Fourier vector
        sqrt_Pspec = math.sqrt(A*(k**(-3.0)))#Square-root of Harrison-Zeldovich Power-spectrum
        khat = np.transpose(norms_and_vecs[norm])# unit Fourier vectors
        
        cos_gamma1 = np.dot(np.array([x,y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the first octant        
        cos_gamma2 = np.dot(np.array([x,-y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 2nd octant: y -> - y        
        cos_gamma3 = np.dot(np.array([-x,-y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 3rd octant: x, y -> -x, -y        
        cos_gamma4 = np.dot(np.array([-x,y,z]),khat) 
        # dot of \hat{x} and \hat{k} in exp(ix.k) in the 4th octant: x -> -x 
        
        prefactor = 2*(np.cos(k*R*cos_gamma1)*Phi_k_dict[norm][0]-np.sin(k*R*cos_gamma1)*Phi_k_dict[norm][1] + \
                    np.cos(k*R*cos_gamma2)*Phi_k_dict[norm][2]-np.sin(k*R*cos_gamma2)*Phi_k_dict[norm][3] + \
                    np.cos(k*R*cos_gamma3)*Phi_k_dict[norm][4]-np.sin(k*R*cos_gamma3)*Phi_k_dict[norm][5] + \
                    np.cos(k*R*cos_gamma4)*Phi_k_dict[norm][6]-np.sin(k*R*cos_gamma4)*Phi_k_dict[norm][7])
        prefactor = np.sum(prefactor) #sum over all angles with fixed nnorm
        
        Phi_x += sqrt_Pspec*prefactor #sum over norms
    return Phi_x

def phi_map_cart_parallel(Nside,nmax,Lsize,alpha,seed):
    np.random.seed(seed) # fixes random seed, for debugging
    normvecs_dict = normsandangles.vector_n(nmax,alpha) #initialize norms and vectors
    randphi_dict = phi_of_k_cart(normvecs_dict) #initialize random values of \phi(\vec{k>)
    
    def phi_of_x_cart_wrapper(xyz):
        x, y, z = xyz
        return phi_of_x_cart(Lsize, x, y, z, normvecs_dict, randphi_dict, alpha)

    n_jobs = multiprocessing.cpu_count()
    phi_in_pixel = Parallel(n_jobs=n_jobs)(delayed(phi_of_x_cart_wrapper)(xyz) for xyz in np.transpose(hp.pix2vec(Nside,np.arange(hp.nside2npix(Nside)))))
    return np.array(phi_in_pixel)

In [48]:
%%time
mapa2 = phi_map_cart_parallel(64,6,0.1,4,1)

CPU times: user 19.2 s, sys: 337 ms, total: 19.5 s
Wall time: 1min 21s


## Checando se os arrays gerados são iguais

In [49]:
def is_array_equal(array1,array2):
    if len(array1) != len(array2):
        return False
    for i in range(len(array1)):
        if(array1[i] != array2[i]):
            return False
    return True

In [50]:
is_array_equal(mapa1,mapa2)

True

In [2]:
pixels = np.arange(hp.nside2npix(12))

In [6]:
pixels[-1]

1727

In [10]:
12*12*12

1728